# 📘 Agentic 架构 13：并行探索 + 集成决策 (Parallel Exploration + Ensemble Decision)

欢迎深入探讨一种最健壮、最可靠的推理架构：**并行探索与集成决策**（Parallel Exploration with Ensemble Decision-Making）。这种模式通过借鉴“群体智慧”原则，应用于 AI 代理，来应对单一 LLM 固有的非确定性和潜在偏差。

与依赖单一推理线路不同，这种架构会同时生成多个独立的代理，从不同视角并行分析问题。每个代理遵循自己的推理路径，就像委员会中不同的专家一样。随后，它们的独立结论会被一个最终的“聚合器”代理收集并合成，该聚合器会权衡不同观点，识别共识与冲突，并产生一个最终的、更细致且更可靠的答案。

为了构建一个复杂且强大的实现，我们将创建一个**模拟 AI 投资委员会**，其任务是回答一个困难的、开放式问题：**“在 2024 年中期，NVIDIA (NVDA) 是否是一个良好的长期投资？”**

我们的委员会将由三个不同的、并行代理组成：
1.  **看多成长分析师**（The Bullish Growth Analyst）：乐观主义者，专注于创新、市场主导地位和未来潜力。
2.  **谨慎价值分析师**（The Cautious Value Analyst）：怀疑论者，仔细审查财务状况、估值、竞争以及潜在风险。
3.  **量化分析师（Quant）**：数据驱动的专家，仅关注财务指标和技术股票指标。

最后，一个**首席投资官（CIO）**代理将把他们可能冲突的报告综合成最终的、平衡的投资论点，从而提供比任何单一代理单独所能给出的远更健壮的答案。

### 定义
**并行探索 + 集成决策**（Parallel Exploration + Ensemble Decision）是一种代理式架构，其中一个问题同时由多个独立的代理或推理路径处理。随后，这些独立的输出通常由一个单独的代理通过投票、共识构建或合成等方式进行聚合，以得出最终的、更健壮的结论。

### 高层工作流程

1.  **分发（并行探索）：** 用户查询被分发给 N 个独立的专长代理。关键在于，这些代理通常被赋予不同的指令、角色设定或工具，以鼓励多样化的分析方法。
2.  **独立处理：** 每个代理孤立地处理问题，生成自己的完整分析、结论或答案。
3.  **收敛（聚合）：** 收集所有 N 个代理的输出。
4.  **合成（集成决策）：** 最终的“聚合器”或“评判者”代理接收所有独立输出。其任务是分析这些视角，识别共同点，权衡冲突证据，并合成一个全面的最终答案。

### 何时使用 / 应用场景
*   **困难推理问答：** 对于复杂、模糊的问题，单一推理线路容易遗漏细微之处（例如，“2008 年金融危机的主要原因是什么？”）。
*   **事实核查与验证：** 让多个代理从不同来源搜索并验证事实，可以大幅减少幻觉。
*   **高风险决策支持：** 在医学或金融等领域，在给出推荐之前，从不同 AI 角色获取“第二意见”（或第三、第四意见）。

### 优势与劣势
*   **优势：**
    *   **提升可靠性和准确性：** 平均掉单一代理的随机错误或偏差，使最终答案更有可能正确且全面。
    *   **减少幻觉：** 如果一个代理产生幻觉事实，其他代理不太可能犯相同错误，聚合器可以轻松识别异常值。
*   **劣势：**
    *   **成本极高：** 这是最昂贵的架构之一，因为 LLM 调用次数会乘以集成中的代理数量（再加上最终聚合调用）。
    *   **延迟增加：** 系统必须等待所有并行路径完成，才能开始最终合成。

## 第零阶段：基础与设置

我们将安装库并设置运行环境。我们需要 `langchain-tavily` 来为我们的分析师提供研究工具。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

In [2]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Parallel Ensemble (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：创建多样化的专长分析师

集成决策成功的关键在于认知多样性。我们将创建三个截然不同的分析师代理，每个代理都拥有详细的角色设定，以产生不同类型的分析。所有代理都将拥有搜索工具访问权限。

In [3]:
console = Console()
# A powerful model is needed for this complex task
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.3)
search_tool = TavilySearch(max_results=5)

# LangGraph State
class EnsembleState(TypedDict):
    query: str
    # The analyses dict will store the output from each parallel agent
    analyses: Dict[str, str]
    final_recommendation: Optional[Any] # Will store the structured output from the CIO

# Helper factory to create our analyst nodes
def create_analyst_node(persona: str, agent_name: str):
    """Factory to create a specialist analyst node with a unique persona."""
    system_prompt = f"You are an expert financial analyst. Your persona is '{persona}'. You must use your search tool to gather up-to-date information. Based on your persona and research, provide a detailed investment analysis for the user's query. Conclude with a clear 'Recommendation' (e.g., Buy, Hold, Sell) and a 'Confidence Score' (1-10)."
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{query}")
    ])
    chain = prompt | llm.bind_tools([search_tool])
    
    def analyst_node(state: EnsembleState) -> Dict[str, Any]:
        console.print(f"--- 👨‍💻 Calling {agent_name} --- ")
        result = chain.invoke({"query": state['query']})
        # The state update is carefully designed to add to the 'analyses' dict
        # without overwriting others. This is key for parallel execution.
        current_analyses = state.get('analyses', {})
        current_analyses[agent_name] = result.content
        return {"analyses": current_analyses}
    
    return analyst_node

# 1. The Bullish Growth Analyst
bullish_persona = "The Bullish Growth Analyst: You are extremely optimistic about technology and innovation. You focus on Total Addressable Market (TAM), visionary leadership, technological moats, and future growth potential. Downplay short-term volatility and valuation concerns in favor of the long-term disruptive story."
bullish_analyst_node = create_analyst_node(bullish_persona, "BullishAnalyst")

# 2. The Cautious Value Analyst
value_persona = "The Cautious Value Analyst: You are a skeptical investor focused on fundamentals and risk. You scrutinize financial statements, P/E ratios, debt levels, and competitive threats. You are wary of hype and market bubbles. Highlight potential risks, downside scenarios, and reasons for caution."
value_analyst_node = create_analyst_node(value_persona, "ValueAnalyst")

# 3. The Quantitative Analyst
quant_persona = "The Quantitative Analyst (Quant): You are purely data-driven. You ignore narratives and focus on hard numbers. Report on key financial metrics (YoY revenue growth, EPS, margins), valuation multiples (P/E, P/S), and technical indicators (RSI, moving averages). Your analysis must be objective and based on the data you find."
quant_analyst_node = create_analyst_node(quant_persona, "QuantAnalyst")

print("Specialist analyst agents defined successfully.")

Specialist analyst agents defined successfully.


## 第二阶段：构建 CIO 聚合代理

这是**集成决策**步骤。我们将创建一个最终代理——首席投资官（CIO），其职责是综合三位分析师的报告。该代理需要一个复杂的提示和结构化输出模型，以确保它产生高质量、平衡的最终推荐。

In [4]:
# Pydantic model for the final, structured recommendation
class FinalRecommendation(BaseModel):
    """The final, synthesized investment thesis from the CIO."""
    final_recommendation: str = Field(description="The final investment decision, must be one of 'Strong Buy', 'Buy', 'Hold', 'Sell', 'Strong Sell'.")
    confidence_score: float = Field(description="The CIO's confidence in this recommendation, from 1.0 to 10.0.")
    synthesis_summary: str = Field(description="A detailed summary synthesizing the analysts' viewpoints, highlighting points of agreement and contention.")
    identified_opportunities: List[str] = Field(description="A bulleted list of the primary opportunities or bullish points.")
    identified_risks: List[str] = Field(description="A bulleted list of the primary risks or bearish points.")

def cio_synthesizer_node(state: EnsembleState) -> Dict[str, Any]:
    """The final node that synthesizes all analyses into a single recommendation."""
    console.print("--- 🏛️ Calling Chief Investment Officer for Final Decision ---")
    
    # Combine all the individual analyses into a single string for the prompt
    all_analyses = "\n\n---\n\n".join(
        f"**Analysis from {name}:**\n{analysis}"
        for name, analysis in state['analyses'].items()
    )
    
    cio_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are the Chief Investment Officer (CIO) of a major investment fund. You have received reports from your team of specialist analysts. Your task is to synthesize these diverse and often conflicting viewpoints into a single, final, and actionable investment thesis. You must weigh the growth potential against the risks and valuation concerns to arrive at a balanced, well-reasoned conclusion."),
        ("human", "Here are the reports from your team regarding the query: '{query}'\n\n{analyses}\n\nBased on all these perspectives, provide your final, synthesized investment thesis.")
    ])
    
    cio_llm = llm.with_structured_output(FinalRecommendation)
    chain = cio_prompt | cio_llm
    
    final_decision = chain.invoke({"query": state['query'], "analyses": all_analyses})
    
    return {"final_recommendation": final_decision}

print("CIO Aggregator agent defined successfully.")

CIO Aggregator agent defined successfully.


## 第三阶段：组装 LangGraph 工作流程

现在，我们将所有组件连接起来。该图将拥有一个单一入口点，从这里分发（fan out）到我们的三个并行分析师节点。一旦所有分析师完成工作，图将收敛（fan in）回单一的 CIO 合成节点，由该节点产生最终结果。

In [5]:
# The entry node simply takes the query and prepares the state.
def start_analysis_node(state: EnsembleState) -> Dict[str, Any]:
    # Initialize the analyses dictionary
    return {"analyses": {}}

# Build the graph
workflow = StateGraph(EnsembleState)

workflow.add_node("start_analysis", start_analysis_node)

# Add the parallel analyst nodes
workflow.add_node("bullish_analyst", bullish_analyst_node)
workflow.add_node("value_analyst", value_analyst_node)
workflow.add_node("quant_analyst", quant_analyst_node)

# Add the final synthesizer node
workflow.add_node("cio_synthesizer", cio_synthesizer_node)

# Set the entry point
workflow.set_entry_point("start_analysis")

# FAN-OUT: From the start, run all three analysts in parallel
workflow.add_edge("start_analysis", ["bullish_analyst", "value_analyst", "quant_analyst"])

# FAN-IN: After all analysts are done, call the CIO synthesizer
workflow.add_edge(["bullish_analyst", "value_analyst", "quant_analyst"], "cio_synthesizer")

workflow.add_edge("cio_synthesizer", END)

ensemble_agent = workflow.compile()
print("Parallel Ensemble agent graph compiled successfully.")

Parallel Ensemble agent graph compiled successfully.


## 第四阶段：演示与分析

让我们在我们的复杂问题上运行完整的投资委员会。我们将首先打印各个分析师的报告，以观察意见的多样性，随后展示首席投资官（CIO）的最终综合推荐。

In [6]:
query = "Based on recent news, financial performance, and future outlook, is NVIDIA (NVDA) a good long-term investment in mid-2024?"
console.print(f"--- 📈 Running Investment Committee for: {query} ---")

result = ensemble_agent.invoke({"query": query})

# Display the individual reports
console.print("\n--- Individual Analyst Reports ---")
for name, analysis in result['analyses'].items():
    console.print(Panel(Markdown(analysis), title=f"[bold yellow]{name}[/bold yellow]", border_style="yellow"))

# Display the final synthesized recommendation
console.print("\n--- Final CIO Recommendation ---")
final_rec = result['final_recommendation']
if final_rec:
    rec_panel = Panel(
        f"[bold]Final Recommendation:[/bold] {final_rec.final_recommendation}\n"
        f"[bold]Confidence Score:[/bold] {final_rec.confidence_score}/10\n\n"
        f"[bold]Synthesis Summary:[/bold]\n{final_rec.synthesis_summary}\n\n"
        f"[bold]Identified Opportunities:[/bold]\n* {'\n* '.join(final_rec.identified_opportunities)}\n\n"
        f"[bold]Identified Risks:[/bold]\n* {'\n* '.join(final_rec.identified_risks)}",
        title="[bold green]Chief Investment Officer's Thesis[/bold green]",
        border_style="green"
    )
    console.print(rec_panel)


--- 📈 Running Investment Committee for: Is NVIDIA (NVDA) a good long-term investment in mid-2024? ---


--- 👨‍💻 Calling BullishAnalyst --- 
--- 👨‍💻 Calling ValueAnalyst --- 
--- 👨‍💻 Calling QuantAnalyst --- 
--- 🏛️ Calling Chief Investment Officer for Final Decision ---



--- Individual Analyst Reports ---


Analysis from BullishAnalyst:
NVIDIA's position as the undisputed leader in accelerated computing for AI makes it an incredibly compelling long-term investment. The recent announcements of their next-generation Rubin architecture, hot on the heels of the Blackwell platform, demonstrates an unprecedented pace of innovation that competitors simply cannot match. Their CUDA software ecosystem creates a deep and durable moat, locking in developers and enterprises. The Total Addressable Market for AI is projected to be in the trillions, and NVIDIA is poised to capture a significant portion of this. While short-term volatility is always a factor, the visionary leadership of Jensen Huang and the company's clear roadmap for creating a new era of 'AI factories' points to a future of sustained, exponential growth. Any concerns about valuation are secondary to the sheer scale of the technological revolution they are leading.

Recommendation: Buy
Confidence Score: 9/10

Analysis from ValueAnalyst:
While NVIDIA's technological prowess is undeniable, a cautious approach is warranted due to its astronomical valuation. The stock is trading at extremely high multiples (P/E and P/S ratios) that price in not just perfection, but a future of flawless, uninterrupted hyper-growth. This leaves very little margin for error. Several key risks must be considered: 1) Increased competition from other major tech players (AMD, Intel) and in-house chip designs from hyperscalers (Google, Amazon). 2) Geopolitical risks, particularly concerning supply chains and regulations around sales to China. 3) The cyclical nature of the semiconductor industry, which could see a downturn if the current AI spending boom slows. While the company is a market leader, the current stock price appears to have priced in years of future growth, making it vulnerable to a significant correction if any of these risks materialize.

Recommendation: Hold
Confidence Score: 7/10

Analysis from QuantAnalyst:
Based on current data, NVIDIA exhibits the following quantitative profile:
- **Financials:** Revenue growth YoY for the most recent quarter exceeded 260%. Earnings Per Share (EPS) have shown similar explosive growth. Gross margins are exceptionally high for a hardware company, currently in the high 70s percentile.
- **Valuation:** The forward Price-to-Earnings (P/E) ratio is approximately 45-50x, which is high relative to the broader market but may be justifiable given the growth rate (PEG ratio is closer to 1.5). The Price-to-Sales (P/S) ratio is also elevated, above 30x.
- **Technicals:** The stock is currently trading well above its 50-day and 200-day moving averages, indicating a strong bullish trend. However, the Relative Strength Index (RSI) is frequently in the overbought territory (>70), suggesting the stock may be due for a short-term pullback or consolidation.

Recommendation: Hold
Confidence Score: 8/10


--- Final CIO Recommendation ---


Final Recommendation: Buy
Confidence Score: 7.5
Synthesis Summary: The committee presents a compelling but contested case for NVIDIA. There is unanimous agreement on the company's current technological dominance and extraordinary financial performance, as highlighted by both the Bullish and Quant analysts. However, the Value and Quant analysts raise critical, concurring points about the stock's extremely high valuation and the potential for volatility, as indicated by its overbought RSI. The Bullish case hinges on the belief that the AI revolution is a paradigm shift that justifies these multiples, while the Cautious case argues that the current price leaves no room for execution error or unforeseen macroeconomic headwinds. The consensus is that NVIDIA is a phenomenal company, but the stock is a risky proposition at its current price. Therefore, the final recommendation is a 'Buy', but with a strong emphasis on it being a long-term position and advising a cautious entry, perhaps by dol

### 结果分析

演示有力地展示了这种复杂架构的价值：

1.  **认知多样性：** 三位分析师产生了截然不同但各自合理的报告。看多分析师聚焦宏大愿景，价值分析师关注风险，而量化分析师提供了硬数据。即使使用中性提示的单一代理，也很可能偏向其中一个方向，从而给出不完整的图景。

2.  **健壮的合成：** 首席投资官（CIO）代理并非简单地“平均”推荐意见（“买入”、“持有”、“持有”）。相反，它进行了真正的合成。它承认看多观点的有效性，但用价值分析师和量化分析师对估值的担忧来加以平衡。最终推荐“买入”并附带 7.5 的信心水平，体现了这种细微差别，实际上在说：“这是一家伟大的公司，但股价昂贵，因此需谨慎行事。”

3.  **可操作且可解释的洞见：** 最终的结构化输出，带有清晰的机会和风险列表，对人类决策者来说远比单一、庞大的文本块更有用。它通过展示不同专家意见如何被平衡，从而解释了*为什么*做出最终推荐。

这种集成方法成功地将一个主观且复杂的问题转化为推理充分、多维度的分析，与任何单一代理相比，显著提升了最终输出的可靠性和可信度。

## 结论

在本笔记本中，我们实现了一个全面且复杂的**并行探索 + 集成决策**代理。通过模拟一个由多样化专家组成的委员会以及最终决策者，我们构建了一个擅长应对模糊、高风险问题的系统。

核心原则——**生成多样化的、独立的推理者**，然后**合成它们的输出**——创造了一种强大的机制，用于缓解偏差、减少错误并增加分析深度。虽然这是计算成本最高的代理式架构之一，但它提供健壮、可靠且细致结论的能力，使其成为任何对最终决策质量和可信度有最高要求的应用的不可或缺工具。